# Thoracic Surgery Data Set



* 470 data points
* 16 features,
* binary classification (1 year survival period, 1=died, 0=not)
* needs to be split manually into train and test --> 70/30 split

source: https://archive.ics.uci.edu/ml/datasets/Thoracic+Surgery+Data


In [63]:
import numpy as np
import sklearn.model_selection

In [64]:
dataset = np.loadtxt(fname='../../datasets/classification/ThoraricSurgery.arff', delimiter=',', skiprows=21, dtype='str')
y = dataset[:, -1] # for last column
X = dataset[:, :-1] # for all but last column

In [65]:
y[y=='F'] = 0
y[y=='T'] = 1

y = np.array(y, dtype='int32')

X[X=='DGN1'] = 1
X[X=='DGN2'] = 2
X[X=='DGN3'] = 3
X[X=='DGN4'] = 4
X[X=='DGN5'] = 5
X[X=='DGN6'] = 6
X[X=='DGN8'] = 8

X[X=='PRZ0'] = 0
X[X=='PRZ1'] = 1
X[X=='PRZ2'] = 2

X[X=='F'] = 0
X[X=='T'] = 1

X[X=='OC11'] = 11
X[X=='OC12'] = 12
X[X=='OC13'] = 13
X[X=='OC14'] = 14

X = np.array(X, dtype='float32')

In [66]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size=0.3, random_state=0)